In [1]:
%run data_clean_and_feature_engineering.ipynb

In [2]:
%run model_tunning.ipynb

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import time as time
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingRandomSearchCV as HRSCV
from sklearn.ensemble import RandomForestRegressor

In [4]:
T=0
start=time.time()
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
total = pd.concat([train, test], axis = 0).reset_index(drop=True)
end=time.time()
print('time:',end-start)
T=T+(end-start)

time: 0.047920942306518555


# data clean

In [5]:
start=time.time()
total=data_clean(total)
end=time.time()
print('time:',end-start)
T=T+(end-start)
total

time: 0.45713210105895996


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1431,60,RL,60.0,21930,Pave,NA,IR3,Lvl,AllPub,...,0,0,0,NA,0,7,2006,WD,Normal,192140
1,757,60,RL,68.0,10769,Pave,NA,IR1,Lvl,AllPub,...,0,0,0,NA,0,4,2009,WD,Normal,212000
2,632,120,RL,34.0,4590,Pave,NA,Reg,Lvl,AllPub,...,0,0,0,NA,0,8,2007,WD,Normal,209500
3,1193,50,RM,60.0,9600,Pave,Grvl,Reg,Lvl,AllPub,...,0,0,0,GdWo,0,7,2007,WD,Normal,125000
4,773,80,RL,94.0,7819,Pave,NA,Reg,Lvl,AllPub,...,0,0,0,MnPrv,0,3,2010,WD,Abnorml,107000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1327,30,RH,70.0,4270,Pave,NA,Reg,Bnk,AllPub,...,0,0,0,NA,0,5,2007,WD,Normal,79000
1456,1154,30,RM,59.5,5890,Pave,NA,Reg,Lvl,AllPub,...,0,0,0,NA,0,6,2008,WD,Normal,120500
1457,665,20,RL,49.0,20896,Pave,NA,IR2,Lvl,AllPub,...,0,0,0,NA,0,1,2006,New,Partial,423000
1458,1259,80,RL,59.0,9587,Pave,NA,IR1,Lvl,AllPub,...,0,0,0,NA,0,7,2008,WD,Normal,190000


# feature engineering

In [6]:
start=time.time()
df=feature_transform(total)
end=time.time()
print('time:',end-start)
T=T+(end-start)
df

time: 0.10679817199707031


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Utilities,Neighborhood,Condition1,HouseStyle,...,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA
0,1431,60,2.0,60.0,104892.652000,1,0,8,35,5,...,1,0,0,0,0,0,0,0,0,1
1,757,60,2.0,68.0,11853.665814,1,0,5,15,5,...,1,0,0,0,0,0,0,0,0,1
2,632,120,2.0,34.0,4027.734205,1,0,16,15,2,...,0,0,0,1,0,0,0,0,0,1
3,1193,50,1.0,60.0,8424.019251,1,0,17,15,0,...,1,0,0,0,0,0,0,0,0,1
4,773,80,2.0,94.0,6861.188180,1,0,7,15,7,...,1,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1327,30,1.1,70.0,3746.933563,1,0,7,15,2,...,1,0,0,0,0,0,0,0,0,1
1456,1154,30,1.0,59.5,5593.547425,1,0,9,15,2,...,1,0,0,0,0,0,0,1,0,0
1457,665,20,2.0,49.0,80218.251429,1,0,21,35,2,...,1,0,0,0,0,0,0,0,0,1
1458,1259,80,2.0,59.0,10552.613442,1,0,8,15,7,...,1,0,0,0,0,0,0,0,0,1


# Train test split

In [7]:
start=time.time()
target = 'SalePrice'
train_size = train.shape[0]
Y = df[target]
X = df.drop(columns = [target])

# training data

x_train = np.array(X[: train_size])
y_train = np.array(Y[:train_size])

# testing data

x_test = np.array(X[train_size: ])
y_test = np.array(Y[train_size: ])

print(f'Training: {x_train.shape, y_train.shape}')
print(f'Testing: {x_test.shape, y_test.shape}')
end=time.time()
T=T+(end-start)
print('time:',end-start)

Training: ((1095, 79), (1095,))
Testing: ((365, 79), (365,))
time: 0.005964040756225586


# anova use F-score to select

In [8]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

start=time.time()
# configure to select all features
fs= SelectKBest(f_classif, k=40)#fvalue_selector

# learn relationship from training data
fs.fit(x_train, y_train)
# transform train input data
X_train_fs = fs.transform(x_train)
# transform test input data
X_test_fs = fs.transform(x_test)

end=time.time()
T=T+(end-start)
print('time:',end-start)

time: 0.04750394821166992


/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


### data clean & feature engineering time

In [9]:
T

0.6653192043304443

# with Autoencoder

In [10]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Dense
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

In [11]:
def build_autoencoder(n_hidden=1, n_neurons=30,lr=0.01,activation='softplus'):
    autoencoder = Sequential()
    autoencoder.add(Input(shape=(n_inputs,),name='input'))
    
    autoencoder.add(Dense(n_inputs*2, activation=activation,name='encoder_1'))
    
    for layer in range(n_hidden):
        if n_hidden>1 and layer==n_hidden-1:
            n_neurons=n_neurons/2
        
        autoencoder.add(Dense(n_neurons, activation=activation,name='encoder_{}'.format(layer+2)))
        
    
    if n_hidden>1:
        n_neurons=n_neurons*2
        for layer in range(n_hidden-1):
            autoencoder.add(Dense(n_neurons, activation=activation,name='decoder_{}'.format(layer+1)))
    
    autoencoder.add(Dense(n_inputs*2, activation=activation,name='decoder_{}'.format(n_hidden+1)))
    autoencoder.add(Dense(n_inputs, activation='linear',name='output'))
    opt=optimizers.Adadelta(lr=lr)
    autoencoder.compile(optimizer=opt,loss='mse')
    
    return autoencoder

### model tunning

In [12]:
start=time.time()

n_inputs=X_train_fs.shape[1]
X_train, X_val, Y_train, y_val = train_test_split(X_train_fs, y_train, test_size=0.3, random_state=1)
# scale input data
trans_in = MinMaxScaler()
trans_in.fit(X_train)
X_train_ = trans_in.transform(X_train)
X_val_ = trans_in.transform(X_val)

model = KerasRegressor(build_fn=build_autoencoder, epochs=100, batch_size=16, verbose=0)

param = {
    "n_hidden": [1],
    "n_neurons": [33,32,31,30,29,28],
    'lr' : [1.0],
    'activation' : ['softplus']
}

grid = GridSearchCV(estimator=model, param_grid=param, scoring= 'neg_mean_absolute_error', cv = 3,n_jobs=4)
grid_result = grid.fit(X_train_,X_train_,validation_data=(X_val_,X_val_))

#summarize results
#print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
display(grid_result.best_params_)
n_hidden = grid_result.best_params_['n_hidden']
n_neurons = grid_result.best_params_['n_neurons']
lr = grid_result.best_params_['lr']
activation = grid_result.best_params_['activation']

end=time.time()
print('time:',end-start)

tun_time=end-start

{'activation': 'softplus', 'lr': 1.0, 'n_hidden': 1, 'n_neurons': 33}

time: 154.91838002204895


### autoencoder train

In [13]:
start=time.time()
autoencoder=build_autoencoder(n_hidden=n_hidden, n_neurons=n_neurons,lr=lr,activation=activation)
autoencoder.fit(X_train_, X_train_, epochs=100, batch_size=16, verbose=2, validation_data=(X_val_,X_val_))
print()
print('autoencoder_model :')
autoencoder.summary()
encoder=Model(autoencoder.input,autoencoder.get_layer('encoder_{}'.format(2)).output)
print()
print('encoder_model :')
encoder.summary()
end=time.time()
print('time:',end-start)

T+=(end-start)

Train on 766 samples, validate on 329 samples
Epoch 1/100
766/766 - 1s - loss: 0.1396 - val_loss: 0.0397
Epoch 2/100
766/766 - 0s - loss: 0.0364 - val_loss: 0.0383
Epoch 3/100
766/766 - 0s - loss: 0.0354 - val_loss: 0.0372
Epoch 4/100
766/766 - 0s - loss: 0.0356 - val_loss: 0.0373
Epoch 5/100
766/766 - 0s - loss: 0.0341 - val_loss: 0.0358
Epoch 6/100
766/766 - 0s - loss: 0.0332 - val_loss: 0.0353
Epoch 7/100
766/766 - 0s - loss: 0.0312 - val_loss: 0.0336
Epoch 8/100
766/766 - 0s - loss: 0.0303 - val_loss: 0.0324
Epoch 9/100
766/766 - 0s - loss: 0.0297 - val_loss: 0.0300
Epoch 10/100
766/766 - 0s - loss: 0.0279 - val_loss: 0.0308
Epoch 11/100
766/766 - 0s - loss: 0.0281 - val_loss: 0.0289
Epoch 12/100
766/766 - 0s - loss: 0.0274 - val_loss: 0.0293
Epoch 13/100
766/766 - 0s - loss: 0.0266 - val_loss: 0.0283
Epoch 14/100
766/766 - 0s - loss: 0.0260 - val_loss: 0.0272
Epoch 15/100
766/766 - 0s - loss: 0.0252 - val_loss: 0.0268
Epoch 16/100
766/766 - 0s - loss: 0.0256 - val_loss: 0.0273
Epo

### transform by encoder

In [14]:
start=time.time()

x_train_ = trans_in.transform(X_train_fs)
x_test_ = trans_in.transform(X_test_fs)
# encode the train data
X_train_encode = encoder.predict(x_train_)
# encode the test data
X_test_encode = encoder.predict(x_test_)

end=time.time()
print('time:',end-start)

T+=(end-start)

time: 0.1292252540588379


In [15]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.ensemble import GradientBoostingRegressor

In [16]:
M=[]
Time=[]

start=time.time()
RF = RandomForestRegressor()

# set searching hyperparameters
search_params = {
    'n_estimators': np.arange(20, 101),
    'max_depth': np.arange(7, 20),
    'ccp_alpha': np.abs(norm(loc = 1.5, scale = 0.5).rvs(size=20)),
    'criterion': ['mae', 'mse'],
    'min_samples_split': np.arange(2, 8),
    'min_impurity_decrease': np.abs(norm(loc = 1, scale = 0.5).rvs(size=30))
}

# set Successive Halving algorithm
SH_search = HRSCV(
    estimator = RF, param_distributions = search_params, n_candidates = 80, 
    factor = 2, resource = 'n_samples', max_resources='auto', min_resources='smallest', 
    aggressive_elimination=False, cv=3, scoring='neg_mean_absolute_error', refit=True,
    return_train_score=True, random_state=None, n_jobs=4, verbose=0
)

SH_search.fit(X_train_encode, y_train)

print(f'Best score is {(-1) * SH_search.best_score_:.4f}')
print(f'Best params is {SH_search.best_params_}')

best_param = SH_search.best_params_.copy()
best_param['n_jobs'] = 4

RF = RandomForestRegressor(**best_param)

RF.fit(X_train_encode, y_train)

pred = RF.predict(X_test_encode)
mape=regression_report(y_test, pred, True)
end=time.time()

print('time:',end-start)

M.append(mape)
Time.append(T+(end-start))

Best score is 22353.6968
Best params is {'n_estimators': 85, 'min_samples_split': 4, 'min_impurity_decrease': 0.1911395810638079, 'max_depth': 9, 'criterion': 'mae', 'ccp_alpha': 1.2385944287957578}
mse = 1279348204.6795
mae = 21834.1644
rmse = 35767.9774
mape = 0.1246
time: 36.323962688446045


In [17]:
start=time.time()

# set searching hyperparameters
#n_estimators莫認為100
#learning_rate莫認為0.1
#max_depth莫認為3
params = {'n_estimators': [100,200,300,400,500],
          'max_depth': [2,1]}
estimator = GradientBoostingRegressor()
# cross validation
folds = 3
model_cv = GridSearchCV(estimator = estimator, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error',  
                        cv = folds, 
                        return_train_score=True,
                        verbose = 0)            
model_cv.fit(X_train_encode, y_train) 
display(model_cv.best_params_)
n_estimators = model_cv.best_params_['n_estimators']
max_depth = model_cv.best_params_['max_depth']

lm_gbr = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=0.1, max_depth=max_depth, random_state=31)
lm_gbr.fit(X_train_encode, y_train) 
pred_gbr = lm_gbr.predict(X_test_encode)

mape=regression_report(y_test, pred_gbr, True)

end=time.time()
print('time:',end-start)

M.append(mape)
Time.append(T+(end-start))

{'max_depth': 2, 'n_estimators': 300}

mse = 1103640984.7605
mae = 20850.3311
rmse = 33221.0925
mape = 0.1198
time: 33.50006031990051


In [18]:
start=time.time()

lm_rfe=LinearRegression()
lm_rfe.fit(X_train_encode,y_train)
pred_li_rfe = lm_rfe.predict(X_test_encode)
mape=regression_report(y_test, pred_li_rfe, True)

end=time.time()
print('time:',end-start)

M.append(mape)
Time.append(T+(end-start))

mse = 1550784504.8112
mae = 22881.3156
rmse = 39380.0013
mape = 0.1245
time: 0.009714841842651367


In [19]:
start=time.time()
params = {'alpha': [0.001, 0.01, 0.1, 1.0,10,100,1000]}
estimator = Ridge()
# cross validation
folds = 5
model_cv = GridSearchCV(estimator = estimator, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error',  
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train_encode,y_train) 
display(model_cv.best_params_)
alpha = model_cv.best_params_['alpha']

ridge = Ridge(alpha=alpha)
ridge.fit(X_train_encode,y_train)
pred_rid = ridge.predict(X_test_encode)
mape=regression_report(y_test, pred_rid, True)

end=time.time()
print('time:',end-start)

M.append(mape)
Time.append(T+(end-start))

Fitting 5 folds for each of 7 candidates, totalling 35 fits


{'alpha': 0.1}

mse = 1562044858.2648
mae = 23384.8841
rmse = 39522.7132
mape = 0.1272
time: 0.11512899398803711


In [20]:
start=time.time()
#alpha莫認為1
params = {'alpha': [0.001, 0.01, 0.1, 1.0,10,100,1000],
          'max_iter':[1000,1500,2000]
         }

estimator = Lasso()
# cross validation
folds = 5
model_cv = GridSearchCV(estimator = estimator, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error',  
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train_encode,y_train) 
display(model_cv.best_params_)
alpha = model_cv.best_params_['alpha']
max_iter = model_cv.best_params_['max_iter']

lasso = Lasso(alpha=alpha,max_iter=max_iter)
lasso.fit(X_train_encode,y_train) 

pred_la = lasso.predict(X_test_encode)
mape=regression_report(y_test, pred_la, True)

end=time.time()
print('time:',end-start)

M.append(mape)
Time.append(T+(end-start))

Fitting 5 folds for each of 21 candidates, totalling 105 fits


/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 297676046336.0, tolerance: 514142240.0
  positive)
/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 451011411968.0, tolerance: 493614560.0
  positive)
/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 477290987520.0, tolerance: 523039648.0
  positive)
/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality 

/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 370738561024.0, tolerance: 528364416.0
  positive)
/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 386966618112.0, tolerance: 507030528.0
  positive)
/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 80545677312.0, tolerance: 514142240.0
  positive)
/Users/andy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality g

{'alpha': 10, 'max_iter': 1500}

mse = 1555998132.3705
mae = 23104.7152
rmse = 39446.1422
mape = 0.1253
time: 2.966141939163208


In [22]:
method=['En + RF','En + GB','En + LR','En + Ridge','En + Lasso']
K={'method':method,
   'mape':M,
  'time(sec)':Time}
result=pd.DataFrame(K)
result

,method,mape,time(sec)
0,En + RF,0.1246,58.555250
1,En + GB,0.1198,55.731348
2,En + LR,0.1245,22.241002
3,En + Ridge,0.1272,22.346416
4,En + Lasso,0.1253,25.197429


In [23]:
tun_time

132.6585178375244

In [15]:
method=['En + RF','En + GB','En + LR','En + Ridge','En + Lasso']
mape=[0.1246,0.1198,0.1245,0.1272,0.1253]
time=[58.555250,55.731348,22.241002,22.346416,25.197429]
score=[]
for i,j in zip(mape,time):
    score.append(100-(0.2*j)+i)
K={'method':method,
   'mape':mape,
  'time(sec)':time,
  'score':score}
result=pd.DataFrame(K)
result

,method,mape,time(sec),score
0,En + RF,0.1246,58.555250,88.413550
1,En + GB,0.1198,55.731348,88.973530
2,En + LR,0.1245,22.241002,95.676300
3,En + Ridge,0.1272,22.346416,95.657917
4,En + Lasso,0.1253,25.197429,95.085814


In [19]:
reg=[0.0961-0.1198,0.1087-0.1246,0.1215-0.1245,0.1215-0.1272,0.1226-0.1253]
reg

[-0.0237,
 -0.015899999999999997,
 -0.0030000000000000027,
 -0.005700000000000011,
 -0.002699999999999994]

In [21]:
for i in reg:
    round(i,4)
K={'method':method,
   'mape':mape,
   'res':reg,
  'time(sec)':time,
  'score':score}
result=pd.DataFrame(K)
result

,method,mape,res,time(sec),score
0,En + RF,0.1246,-0.0237,58.555250,88.413550
1,En + GB,0.1198,-0.0159,55.731348,88.973530
2,En + LR,0.1245,-0.0030,22.241002,95.676300
3,En + Ridge,0.1272,-0.0057,22.346416,95.657917
4,En + Lasso,0.1253,-0.0027,25.197429,95.085814
